In [1]:
import numpy as np
from ipywidgets import interact, HTML
import random
import pdir


from sidepanel import SidePanel

import ipywidgets as widgets

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *


from regulus.topo import morse_smale, Regulus
from regulus.data.data import Data
from regulus.measures import *
from regulus.models import *
import pandas as pd

In [2]:
samples = 200

# from test_functions import *
# test_function = gerber_different_heights
# X = np.random.rand(samples,2)
# Y = test_function(X)
# all_pts = np.hstack((X,np.expand_dims(Y, axis=1)))

# myfun = test_function
from Hartmann import create, calc_Hartmanny

all_pts = create(samples)
myfun = calc_Hartmanny



In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from refine_topo import Refine
from resample import Resample


def create_reg(all_pts, k=8):
    cols = ['x'+str(i) for i in range(all_pts.shape[1])]

    cols[-1] = 'y'

    df = pd.DataFrame(all_pts, columns=cols)

    pts = Data.from_pts(df)

    # Normalize here  
    
    #pts.normalize()

    print(pts)
    regulus = morse_smale(pts, knn=k)#**kwargs)

    regulus.add_attr('linear', linear_model)
    regulus.add_attr('fitness', fitness)
    regulus.add_attr('relative_fitness', relative_fitness)

    regulus.tree.add_attr('parent_fitness', parent_fitness)
    regulus.tree.add_attr('child_fitness', child_fitness)
    regulus.tree.add_attr('size', node_size)
    regulus.tree.add_attr('rel_size', node_relative_size)
    regulus.tree.add_attr('span', node_span)
    return regulus



def mymeasure(node):
    attr="id"
    if hasattr(node, attr) and hasattr(node,'parent') and hasattr(node.parent,attr):
        cid, pid = getattr(node, attr), getattr(node.parent, attr)
        if cid >=0 and pid>=0: #check both are valid partitions

            cx, cy = node.data.x.values, node.data.y.values
            px, py = node.parent.data.x.values, node.parent.data.y.values
            
            # Linear coeff for both and similarity 
            creg, preg = LinearRegression().fit(cx, cy), \
                        LinearRegression().fit(px, py)

            # fitness
            cscore, pscore = creg.score(cx, cy), preg.score(px, py)
            # sensitivity
            ccoef, pcoef = creg.coef_, preg.coef_
            # intercept
            cint, pint = creg.intercept_, preg.intercept_

            return cosine_similarity(ccoef.reshape(1, -1), pcoef.reshape(1, -1))[0][0]
    return -1  


def regenerate(all_pts, partitions, test_function, pids, step=5):
    dims = all_pts.shape[1]-1
    new_x = np.array([], dtype=np.float64).reshape(0,dims)
    r = Resample()
    
    for ind in pids:
        cur_x = partitions[ind].x.values
        # print("CURX = ", cur_x)
        cur_y = partitions[ind].y.values.reshape(-1,1)
        # print("CURY = ", cur_y)

        cur_p = np.hstack((cur_x, cur_y))
        
        r.resample_data(cur_p, step)
        out = r.newsamples
        new_x = np.vstack((new_x, out))
    new_y = test_function(new_x).reshape(-1,1)
    new_pts = np.hstack((new_x,new_y))
    all_pts2 = np.vstack((all_pts, new_pts))
    return all_pts2


In [6]:
regulus = create_reg(all_pts, k = 10)

partitions = list(regulus.partitions())

rt = Refine(0.5, tree=regulus.tree, traverse="BFS", measure=mymeasure, compare="less", return_attr=lambda x:x.id)

pids = rt.search()

print("Partitions to resample", pids)

all_pts = regenerate(all_pts, partitions, myfun, pids, step=5)

E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


found 9 singles
len(idx)= 278
Partitions to resample [88, 58, 87, 57, 71, 86, 54, 48, 41, 47, 38, 35, 32, 30, 29, 28]


In [10]:
def process(regulus):
    from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget

    tree = TreeWidget(regulus.tree)
    data = DataWidget(data=regulus)
    return tree, data


def draw_tree(tree):
    treev = show_tree(tree)
    w = widgets.Dropdown(
        options=['span', 'fitness', 'parent_fitness', 'child_fitness'],
        value='span',
        description='Attribute:',
        disabled=False,
    )
    panel = SidePanel(title='Fitness')#, side='split-bottom', ref='SidePanel')
    with panel:
        display(w, treev.view, treev.filter)
    return treev


def draw_details(data):
    details = DetailsView(data=data)

    p7 = SidePanel(title='details')
    with p7:
        display(details)
    return details

def link_views(treev, details):
    widgets.link((treev.view, 'details'), (details, 'show'))

In [11]:
tree, data = process(regulus)
treev = draw_tree(tree)
details = draw_details(data)
link_views(treev, details)


TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.3749545336451027, 1: 0.33219917261226684, 2:…

In [72]:
details.show = list(map(lambda x: x, pids))


# Create resample class, resampling a user specified partition, for User-guided resampling

In [27]:
sel_x = partitions[0].x.values

sel_y = partitions[0].y.values.reshape(-1,1)

sel_p = np.hstack((sel_x, sel_y))

r = Resample(sampleAttr="domain")
r.add_pts(sel_p)
r.build()

# The following will be changed later st cyplot works for numpy array
cols = ['x'+str(i) for i in range(sel_p.shape[1])]

cols[-1] = 'y'

df = pd.DataFrame(sel_p, columns=cols)

sel_p = df





# A line chart for a specified partition, different interactions could be added 

In [28]:
from cyplot import plot
myp = plot(sel_p,index = 'y',debug=True, dims=['x0','x1','y'])#ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp.enable(['brush_x','brush'])
myp.on('brush_x', r.change_range)
myp.on('brush', r.change_range)

display(myp)

In [17]:
# from cyplot import plot
myp2 = plot(sel_p,index = 'y',debug=True, ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp2.enable(['brush_x','brush'])
myp2.on('brush_x', r.cb)
myp2.on('brush', r.cb)
display(myp2)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [31]:
myp2.enable('x_brush')
display(myp2)

# random generate pts based on user selected range of function value

In [32]:
r.add_samples(5) # user's specified # of samples


# When user adds enough samples, they could use regression to predict the point locations to resample 

In [33]:
# Would scale back if needed 
new_x = r.get_input()
new_y = test_function(new_x).reshape(-1,1)
new_pts = np.hstack((new_x,new_y))



In [34]:
all_pts = np.vstack((all_pts, new_pts))